<a href="https://colab.research.google.com/github/lgustavo95/CapacitaBR-CienDados/blob/main/QA_MANUAL2_0_REV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1 - Instalação das dependências

In [ ]:
# Atualiza os pacotes do sistema
!apt update

# Instala Tesseract com suporte a português
!apt install -y tesseract-ocr tesseract-ocr-por

# Instala bibliotecas Python necessárias
!pip install pymupdf transformers sentence-transformers torch pytesseract pillow nltk tqdm chromadb

# Instala pdf slumber
!pip install pdfplumber reportlab ipywidgets
!apt-get install -y ttf-mscorefonts-installer  # Arial


# Define o caminho do tessdata para o pytesseract funcionar corretamente
import os
os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata"

# ⚠️ Apaga instância anterior do ChromaDB se necessário
import shutil
if os.path.exists("/content/chroma_db"):
    shutil.rmtree("/content/chroma_db")






Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,036 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,572 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 P

2 - Imports e configurações iniciais

In [ ]:
# Imports padrão
import os
import warnings
import pickle
from itertools import chain  # (manter se for usar depois)

# Imports de bibliotecas externas
import fitz  # PyMuPDF
print("PyMuPDF instalado e funcionando!")
import numpy as np
from PIL import Image
import pytesseract
from transformers import pipeline, AutoTokenizer
from sentence_transformers import SentenceTransformer
from nltk.tokenize.punkt import PunktSentenceTokenizer
from tqdm import tqdm
import nltk
import pdfplumber
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import tempfile

from IPython.display import display, clear_output
import ipywidgets as widgets

# Import específico para Colab
from google.colab import files

# Configurações iniciais
nltk.download('punkt')
warnings.filterwarnings("ignore")
os.environ["TESSDATA_PREFIX"] = "/usr/share/tesseract-ocr/4.00/tessdata"

# Carrega o modelo de Perguntas e Respostas (QA) - em português
qa_pipeline = pipeline(
    "question-answering",
    model="pierreguillou/bert-base-cased-squad-v1.1-portuguese",
    tokenizer="pierreguillou/bert-base-cased-squad-v1.1-portuguese"
)

# Carrega o modelo de embeddings para similaridade semântica
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

print("Modelos carregados com sucesso!")


PyMuPDF instalado e funcionando!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


config.json:   0%|          | 0.00/862 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelos carregados com sucesso!


3 - Funções de extração e pré-processamento

In [ ]:
# 3. PRÉ-PROCESSAMENTO E DIVISÃO EM CHUNKS

def preprocess_text(text):
    """
    Remove quebras de linha, espaços extras e caracteres estranhos do texto.
    """
    import re
    # Remove quebras de linha e múltiplos espaços
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)

    # Remove caracteres não ASCII (opcional, ajuste conforme necessidade)
    # text = re.sub(r'[^\x00-\x7F]+',' ', text)

    return text.strip()

def chunk_text(text, max_length=500):
    """
    Divide texto em chunks (pedaços) com tamanho máximo max_length,
    tentando quebrar em pontos naturais (ponto final) para manter sentido.
    """
    import re

    chunks = []
    start = 0
    text_length = len(text)

    while start < text_length:
        end = min(start + max_length, text_length)

        # Tenta achar o último ponto final antes do limite para não cortar no meio da frase
        if end < text_length:
            last_period = text.rfind('.', start, end)
            if last_period != -1 and last_period > start:
                end = last_period + 1

        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)

        start = end

    return chunks

# Exemplo rápido:
# texto = "Aqui está um exemplo. Vamos dividir este texto em pedaços menores. Isso é útil para o modelo."
# chunks = chunk_text(preprocess_text(texto), max_length=50)
# print(chunks)


4 - Função de chunking

In [ ]:
# 4. DIVIDIR TEXTO EM CHUNKS BASEADO EM SENTENÇAS E TOKENS (COM OVERLAP)

import re
from transformers import AutoTokenizer
from itertools import chain

def split_text_into_sentences(text):
    """
    Divide o texto em sentenças simples baseando-se em pontos finais, exclamação e interrogação.
    """
    sentence_endings = re.compile(r'(?<=[.!?])\s+')
    sentences = sentence_endings.split(text.strip())
    return [s.strip() for s in sentences if s.strip()]

def chunk_text_by_sentences(text, model_name, max_tokens=400, overlap=0.2):
    """
    Divide texto em chunks com até max_tokens tokens, respeitando sentenças.
    Aplica overlap percentual para manter contexto entre chunks.

    Args:
      text (str): Texto completo a ser dividido.
      model_name (str): Nome do modelo Hugging Face para carregar tokenizer.
      max_tokens (int): Máximo de tokens por chunk.
      overlap (float): Percentual (0-1) de sentenças para repetir entre chunks.

    Returns:
      List[str]: Lista de chunks de texto.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    sentences = split_text_into_sentences(text)

    chunks = []
    current_chunk = []
    current_len = 0
    i = 0

    while i < len(sentences):
        sentence = sentences[i]
        tokenized = tokenizer.tokenize(sentence)
        token_len = len(tokenized)

        # Verifica se adicionando esta sentença excede o limite de tokens
        if current_len + token_len > max_tokens:
            if current_chunk:
                # Converte tokens em string para formar chunk
                chunk_text = tokenizer.convert_tokens_to_string(list(chain.from_iterable(current_chunk)))
                if current_len > 20:  # evita chunks muito curtos
                    chunks.append(chunk_text)

                # Define overlap: repetir últimas sentenças no próximo chunk
                overlap_len = int(len(current_chunk) * overlap)
                current_chunk = current_chunk[-overlap_len:] if overlap_len > 0 else []
                current_len = sum(len(sent) for sent in current_chunk)
            else:
                # Se uma sentença sozinha é maior que max_tokens, adiciona diretamente
                chunks.append(tokenizer.convert_tokens_to_string(tokenized))
                i += 1
                current_chunk = []
                current_len = 0
        else:
            current_chunk.append(tokenized)
            current_len += token_len
            i += 1

    # Adiciona o último chunk remanescente
    if current_chunk and current_len > 20:
        chunk_text = tokenizer.convert_tokens_to_string(list(chain.from_iterable(current_chunk)))
        chunks.append(chunk_text)

    return chunks

# Exemplo de uso:
# model_name = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"
# texto_preprocessado = preprocess_text(texto_extraido)
# chunks = chunk_text_by_sentences(texto_preprocessado, model_name=model_name, max_tokens=400, overlap=0.2)
# print(f"Número de chunks gerados: {len(chunks)}")




4.1 - Carregando tokenizer do modelo pierreguillou/bert-base-cased-squad-v1.1-portuguese

In [ ]:
# 4.1 - Carregando tokenizer do modelo pierreguillou/bert-base-cased-squad-v1.1-portuguese

from transformers import AutoTokenizer

# Nome do modelo Hugging Face
model_name = "pierreguillou/bert-base-cased-squad-v1.1-portuguese"

# Carrega o tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Exemplo de texto em português
sample_text = "Qual é a capital do Brasil?"

# Tokeniza o texto
tokens = tokenizer.tokenize(sample_text)

print(f"Tokens: {tokens}")


Tokens: ['Qual', 'é', 'a', 'capital', 'do', 'Brasil', '?']


5 - Classe principal com integração do Chroma DB

In [ ]:
import fitz  # PyMuPDF
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import chromadb
from transformers import pipeline, AutoTokenizer



class ManualQA:
    def __init__(self):
        self.embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
        self.qa_pipeline = pipeline(
            "question-answering",
            model="pierreguillou/bert-large-cased-squad-v1.1-portuguese",
            tokenizer="pierreguillou/bert-large-cased-squad-v1.1-portuguese"
        )
        self.tokenizer = AutoTokenizer.from_pretrained("pierreguillou/bert-large-cased-squad-v1.1-portuguese")
        self.models_loaded = True
        self.collection = None
        self.client = chromadb.Client()  # inicia cliente ChromaDB

    def process_manual(self, pdf_path):
        """
        Extrai texto do PDF, divide em chunks, gera embeddings e indexa no ChromaDB.
        """
        print("Processando manual... isso pode levar alguns minutos dependendo do tamanho do PDF.")
        doc = fitz.open(pdf_path)

        texto_completo = ""
        for page in tqdm(doc, desc="Extraindo texto do PDF"):
            texto_completo += page.get_text().strip() + "\n"

        max_tokens = 400

        def split_text_into_sentences(text):
            import re
            sentence_endings = re.compile(r'(?<=[.!?])\s+')
            return [s.strip() for s in sentence_endings.split(text) if s.strip()]

        sentences = split_text_into_sentences(texto_completo)

        chunks = []
        current_chunk = []
        current_len = 0
        for sentence in sentences:
            token_len = len(self.tokenizer.tokenize(sentence))
            if current_len + token_len > max_tokens:
                chunks.append(" ".join(current_chunk))
                current_chunk = [sentence]
                current_len = token_len
            else:
                current_chunk.append(sentence)
                current_len += token_len
        if current_chunk:
            chunks.append(" ".join(current_chunk))

        embeddings = self.embed_model.encode(chunks, convert_to_numpy=True)

        if "manual_qa_collection" in [col.name for col in self.client.list_collections()]:
            self.client.delete_collection("manual_qa_collection")
        self.collection = self.client.create_collection("manual_qa_collection")

        self.collection.add(
            documents=chunks,
            embeddings=embeddings,
            ids=[str(i) for i in range(len(chunks))]
        )
        print("Manual processado e indexado com sucesso!")

    def ask_question(self, question):
        """
        Recebe uma pergunta, busca contextos relevantes no índice e responde via pipeline de QA.
        """
        if not self.models_loaded or not self.collection:
            raise RuntimeError("Modelos não carregados ou manual não processado.")

        try:
            question_embedding = self.embed_model.encode([question], convert_to_numpy=True)[0]
            results = self.collection.query(
                query_embeddings=[question_embedding],
                n_results=10
            )

            documents = results.get('documents', [[]])
            if not documents or len(documents[0]) == 0:
                return {"answer": "Resposta não encontrada no manual.", "confidence": 0.0}

            melhores_chunks = [chunk.strip() for chunk in documents[0] if chunk.strip()]
            contexto = "\n\n---\n\n".join(melhores_chunks)

            resposta = self.qa_pipeline({
                'question': question,
                'context': contexto
            })
            return {
                "answer": resposta.get('answer', 'Resposta não encontrada.'),
                "confidence": resposta.get('score', 0.0),
                "context": contexto
            }

        except Exception as e:
            print(f"Erro ao processar pergunta: {e}")
            return {"answer": "Erro interno", "confidence": 0.0}


6.1 - Pré processamento para normalização do arquivo pdf

In [ ]:
import textwrap

def processar_pdf_original(caminho_pdf, caminho_saida):
    texto_extraido = ""
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            texto_extraido += pagina.extract_text() + "\n"

    try:
        pdfmetrics.registerFont(TTFont('Arial', '/usr/share/fonts/truetype/msttcorefonts/Arial.ttf'))
        fonte = "Arial"
    except:
        fonte = "Helvetica"
        print("⚠️ Fonte Arial não encontrada. Usando Helvetica.")

    c = canvas.Canvas(caminho_saida, pagesize=A4)
    c.setFont(fonte, 10)

    linhas = [l for l in texto_extraido.split('\n') if l.strip()]  # remove linhas vazias
    largura, altura = A4
    y = altura - 50

    max_width_chars = 100  # ajuste para quebra de linhas longas

    for linha in linhas:
        # quebra linhas muito longas em pedaços menores
        for trecho in textwrap.wrap(linha, max_width_chars):
            if y < 50:
                c.showPage()
                c.setFont(fonte, 10)
                y = altura - 50
            c.drawString(50, y, trecho)
            y -= 12

    c.save()
    return caminho_saida


6 – Upload do PDF e processamento




In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

# Widget para upload do PDF
upload_pdf = widgets.FileUpload(
    accept='.pdf',
    multiple=False,
    description='Upload PDF'
)

btn_processar = widgets.Button(
    description='Processar PDF',
    button_style='success'
)

output = widgets.Output()

# Instância do sistema QA - altere para a classe correta que você estiver usando
qa = ManualQA()

def on_processar_click(b):
    with output:
        clear_output()
        if not upload_pdf.value:
            print("Por favor, faça upload do arquivo PDF primeiro.")
            return

        for nome_arquivo in upload_pdf.value:
            conteudo = upload_pdf.value[nome_arquivo]['content']
            caminho_pdf = f"/content/{nome_arquivo}"
            with open(caminho_pdf, 'wb') as f:
                f.write(conteudo)
            print(f"Arquivo '{nome_arquivo}' salvo com sucesso.")

        # Caminho do arquivo normalizado que será gerado
        caminho_pdf_normalizado = "/content/manual_gol.pdf"

        print("Gerando PDF normalizado com Arial 10...")
        try:
            processar_pdf_original(caminho_pdf, caminho_pdf_normalizado)
            print(f"PDF normalizado salvo em: {caminho_pdf_normalizado}")
        except Exception as e:
            print(f"Erro ao normalizar o PDF: {e}")
            return

        print("Processando manual (indexação e criação do modelo)... isso pode levar alguns minutos.")
        try:
            qa.process_manual(caminho_pdf_normalizado)  # usa o PDF normalizado
            print("Manual processado com sucesso! Agora você pode fazer perguntas.")
        except Exception as e:
            print(f"Erro no processamento do manual: {e}")


btn_processar.on_click(on_processar_click)

display(upload_pdf, btn_processar, output)



Device set to use cpu


FileUpload(value={}, accept='.pdf', description='Upload PDF')

Button(button_style='success', description='Processar PDF', style=ButtonStyle())

Output()

7 - Pergunta

In [ ]:
from IPython.display import clear_output
import ipywidgets as widgets

pergunta_text = widgets.Text(
    value='',
    placeholder='Digite sua pergunta aqui',
    description='Pergunta:',
    disabled=False,
    layout=widgets.Layout(width='70%')
)

btn_perguntar = widgets.Button(
    description='Fazer pergunta',
    button_style='info'
)

output = widgets.Output()

# GIF animado de loading (você pode trocar a URL para outro GIF se quiser)
loading_gif = widgets.HTML(
    value="<img src='https://i.gifer.com/ZZ5H.gif' width='50' height='50'>"
)

def on_perguntar_click(b):
    with output:
        clear_output()
        pergunta = pergunta_text.value.strip()
        if not pergunta:
            print("Digite uma pergunta para continuar.")
            return

        # Verifica se o manual foi processado
        if not (hasattr(qa, 'collection') and qa.collection is not None):
            print("Por favor, processe o manual antes de fazer perguntas.")
            return

        display(loading_gif)  # mostra o loading

        try:
            resposta = qa.ask_question(pergunta)
            clear_output()  # limpa o loading
            print(f"Pergunta: {pergunta}")
            print(f"Resposta: {resposta.get('answer', 'Sem resposta')}")
            print(f"Confiança: {resposta.get('confidence', 0):.2f}")
            print(f"\n[Contexto usado]\n{resposta.get('context', 'Sem contexto disponível')}")
        except Exception as e:
            clear_output()
            print(f"Erro ao responder pergunta: {e}")

btn_perguntar.on_click(on_perguntar_click)

display(pergunta_text, btn_perguntar, output)


Pergunta: Qual a profundidade mínima do perfil do pneu?
Resposta: .
Confiança: 0.25

[Contexto usado]
Rotação e seta Identificação do sentido de rodagem do pneu → Página 238. 8
OU: Outside Identificação do lado externo do pneu → Página 238. MAX INFLATION 350 KPA Limitação para a pressão de ar máxima. 9
(51 psi / 3,51 bar)
M+S ou M/S ou I Indicação para pneus adequados para o inverno (pneus para
10 lama e para neve). Pneus com cravos são identificados de-
pois do S com um E. TWI Indica a posição do indicador de desgaste (Tread Wear Indi-
11
cator) → Página 233. 12 Nome da marca, logotipo Fabricante. 13 Feito na Alemanha País de fabricação. I Identificação específica para a China (China Compulsory
14
Certification). 15 I 023 Selo de identificação do INMETRO. E4 e4 0200477-b Identificação segundo prescrições internacionais com nú-
mero do país emissor da aprovação. Pneus aprovados con-
16 forme o regulamento ECE são identificados com E, pneus
conforme o regulamento EG com e. Em seguida, s